In [3]:
!pip uninstall llama-index -y
!pip install chromadb pydantic typing-extensions
!pip install llama-index-core llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-readers-web llama-index-vector-stores-chroma

Found existing installation: llama-index 0.12.21
Uninstalling llama-index-0.12.21:
  Successfully uninstalled llama-index-0.12.21
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (252 bytes)
  Using cached fastapi-0.115.9-py3-none-any.whl.metadata (27 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached grpcio-1.71.0-cp312-cp312-macosx_10_14_universal2.whl.metadata (3.8 kB)
  Using cached bcrypt-4.3.0-cp39-abi3-macosx_10_12_universal2.whl.metadata (10 kB)
  Using cached kubernetes-32.0.1-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached mmh3-5.1.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (16 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached durationpy-0.9-py3-none-any.whl.metadata (338 by

In [6]:
from typing import List, Dict, Any
from pydantic import BaseModel
import asyncio
import os

from llama_index.core import VectorStoreIndex, Document
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.text_splitter import TokenTextSplitter

# Initialize LLM (Ollama with Llama3.2)
llm = Ollama(model="llama3.2:3b", request_timeout=120.0)

# Define search tool
def search_tool(query: str) -> str:
    return f"Searching: {query}"

tools = [
    FunctionTool.from_defaults(
        fn=search_tool,
        name="Search",
        description="Tool to give respective role features and responsibilities."
    )
]

# Task description
task = (
    "I want to develop, test, and deploy a web and AI-based application for our retail grocery store where customers can "
    "browse, search items, scan items using barcode, add to cart, make online payments, arrange online delivery, track orders, "
    "and provide feedback. Use Scrum-agile with sprints. Provide estimates for each phase separately, covering customer, "
    "project manager, requirements, developers, QA, BAs, DevOps, architects, Scrum Master, security, and technical writers. "
    "Responses should be ≤200 words."
)

# Create documents for indexing
documents = [Document(text=task)]

# Create index
index = VectorStoreIndex.from_documents(documents)

# Define state structure
class AgentState(BaseModel):
    messages: List[Dict[str, str]] = []
    current_role: str = "initiator"

# Role-specific templates
templates = {
    "customer": """
    As a retail store manager, transform requirements into clear IT specifications:
    Develop a web and AI-based app for browsing, searching, barcode scanning, cart management, online payments, delivery, order tracking, and feedback.
    Provide requirements to the product owner.
    """,
    "product_owner": """
    As Product Owner, prioritize backlog and define sprint-wise features:
    - List sprints, tasks, story points.
    - Create user stories, acceptance criteria.
    - Provide sprint-wise user story table.
    Total days: 10.
    """,
    "scrum_master": """
    As Scrum Master, plan sprints and ensure Scrum process:
    - Define sprint features, tasks, story points.
    - Estimate sprint durations (days).
    - Provide sprint-wise feature table.
    Total days: 8.
    """,
    "requirement_engineer": """
    As Requirement Engineer, analyze and document:
    - Sprint-wise features.
    - Estimate pages (10/sprint), productivity (2 pages/hour).
    - List detailed features.
    Total days: 12.
    """,
    "business_analyst": """
    As Business Analyst, create user stories and process flows:
    - Sprint-wise features.
    - Estimate tasks (5/sprint), productivity (1 task/hour).
    - Define user stories, acceptance criteria.
    Total days: 10.
    """,
    "architect": """
    As Architect, design system:
    - Sprint-wise features.
    - Microservices, Python/Django, PostgreSQL, AWS.
    - Estimate pages (8/sprint), productivity (1.5 pages/hour).
    Total days: 15.
    """,
    "ui_ux_designer": """
    As UI/UX Designer, create front-end:
    - Sprint-wise features.
    - Estimate tasks (6/sprint), productivity (1 task/hour).
    - Design material UI, navigation.
    Total days: 12.
    """,
    "software_engineer": """
    As Software Engineer, develop app:
    - Sprint-wise features.
    - Estimate SLOC (1000/sprint), productivity (50 SLOC/hour).
    - Use Django, microservices.
    Total days: 20.
    """,
    "devops_engineer": """
    As DevOps Engineer, set CI/CD:
    - Sprint-wise features.
    - Estimate tasks (4/sprint), productivity (0.8 tasks/hour).
    - Use Jenkins, AWS.
    Total days: 10.
    """,
    "security_engineer": """
    As Security Engineer, secure system:
    - Sprint-wise features.
    - Estimate tasks (3/sprint), productivity (0.5 tasks/hour).
    - Integrate security tests.
    Total days: 8.
    """,
    "testing_engineer": """
    As Testing Engineer, create test cases:
    - Sprint-wise features.
    - Estimate test cases (20/sprint), productivity (4 cases/hour).
    Total days: 10.
    """,
    "documentation_engineer": """
    As Technical Writer, document project:
    - Sprint-wise features.
    - Estimate pages (12/sprint), productivity (2 pages/hour).
    - Provide estimate table.
    Total days: 8.
    """
}

# Query engine for each role
async def process_role(state: AgentState, role: str) -> str:
    query_engine = RetrieverQueryEngine.from_args(index.as_retriever(), llm=llm)
    query = f"{templates[role]}\nLast message: {state.messages[-1]['content'] if state.messages else task}"
    response = await query_engine.aquery(query)
    state.messages.append({"role": role, "content": str(response)})
    return str(response)

# Streaming workflow
async def run_streaming_workflow():
    state = AgentState(messages=[{"role": "initiator", "content": task}], current_role="initiator")
    roles = [
        "customer", "product_owner", "scrum_master", "requirement_engineer", "business_analyst",
        "architect", "ui_ux_designer", "software_engineer", "devops_engineer", "security_engineer",
        "testing_engineer", "documentation_engineer"
    ]

    print("Starting Agile Scrum Workflow...\n")
    print("="*50 + "\n")

    for role in roles:
        print(f"\n{role.upper()} OUTPUT:\n")
        print("-"*50)
        response = await process_role(state, role)
        print(response)
        print(f"\n\n--- Role Transition: {role} → {roles[roles.index(role) + 1] if role != roles[-1] else 'Complete'} ---\n")

# Execute workflow
import nest_asyncio
nest_asyncio.apply()
asyncio.run(run_streaming_workflow())

ModuleNotFoundError: No module named 'llama_index'